In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_inline
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import recall_score, confusion_matrix, classification_report,f1_score,precision_score,roc_auc_score
from xgboost import XGBClassifier
%matplotlib inline

In [179]:
df = pd.read_csv('train.csv')
df 

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [180]:
df = df[["patient_id", "image_id", "laterality", "view", "age",'implant','machine_id','biopsy' ,"cancer"]]

In [181]:
df.head()

,patient_id,image_id,laterality,view,age,implant,machine_id,biopsy,cancer
0,10006,462822612,L,CC,61.0,0,29,0,0
1,10006,1459541791,L,MLO,61.0,0,29,0,0
2,10006,1864590858,R,MLO,61.0,0,29,0,0
3,10006,1874946579,R,CC,61.0,0,29,0,0
4,10011,220375232,L,CC,55.0,0,21,0,0


In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  54706 non-null  int64  
 1   image_id    54706 non-null  int64  
 2   laterality  54706 non-null  object 
 3   view        54706 non-null  object 
 4   age         54669 non-null  float64
 5   implant     54706 non-null  int64  
 6   machine_id  54706 non-null  int64  
 7   biopsy      54706 non-null  int64  
 8   cancer      54706 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 3.8+ MB


In [183]:
df.describe()

,patient_id,image_id,age,implant,machine_id,biopsy,cancer
count,54706.000000,5.470600e+04,54669.000000,54706.000000,54706.000000,54706.000000,54706.000000
mean,32698.865262,1.079386e+09,58.543928,0.026999,54.618378,0.054272,0.021168
std,18893.861534,6.183269e+08,10.050884,0.162081,44.784800,0.226556,0.143944
min,5.000000,6.849100e+04,26.000000,0.000000,21.000000,0.000000,0.000000
25%,16481.000000,5.458153e+08,51.000000,0.000000,29.000000,0.000000,0.000000
50%,32432.000000,1.082689e+09,59.000000,0.000000,49.000000,0.000000,0.000000
75%,48999.000000,1.613228e+09,66.000000,0.000000,49.000000,0.000000,0.000000
max,65534.000000,2.147472e+09,89.000000,1.000000,216.000000,1.000000,1.000000


laterality:

    0: L (Left)
    1: R (Right)

view:

    0: AT (axillary tail)   
    1: CC (craniocaudal)
    2: LM (latero-medial)
    3: LMO (latero-medial oblique)
    4: ML (medio-lateral)
    5: MLO (mediolateral oblique)

In [184]:
le_laterality = LabelEncoder()
le_view = LabelEncoder()

df['laterality'] = le_laterality.fit_transform(df['laterality'])
df['view'] = le_view.fit_transform(df['view'])

df.head()


/var/folders/tr/1ykjgs6x0dlg5g_7jm6k81qr0000gn/T/ipykernel_48960/3923209517.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['laterality'] = le_laterality.fit_transform(df['laterality'])
/var/folders/tr/1ykjgs6x0dlg5g_7jm6k81qr0000gn/T/ipykernel_48960/3923209517.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['view'] = le_view.fit_transform(df['view'])


,patient_id,image_id,laterality,view,age,implant,machine_id,biopsy,cancer
0,10006,462822612,0,1,61.0,0,29,0,0
1,10006,1459541791,0,5,61.0,0,29,0,0
2,10006,1864590858,1,5,61.0,0,29,0,0
3,10006,1874946579,1,1,61.0,0,29,0,0
4,10011,220375232,0,1,55.0,0,21,0,0


In [185]:
df['age'] = df['age'].fillna(59)


/var/folders/tr/1ykjgs6x0dlg5g_7jm6k81qr0000gn/T/ipykernel_48960/1958681743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df['age'].fillna(59)


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54706 entries, 0 to 54705
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  54706 non-null  int64  
 1   image_id    54706 non-null  int64  
 2   laterality  54706 non-null  int64  
 3   view        54706 non-null  int64  
 4   age         54706 non-null  float64
 5   implant     54706 non-null  int64  
 6   machine_id  54706 non-null  int64  
 7   biopsy      54706 non-null  int64  
 8   cancer      54706 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 3.8 MB


In [195]:
X, y = df.iloc[:,:-1], df.iloc[:,-1]


In [196]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [189]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,stratify=y_train, random_state=1337)

In [190]:
eval_set=[(X_train,y_train),(X_val,y_val)]
weights = 46
model = XGBClassifier(
                          scale_pos_weight = weights,
                          n_jobs = 4,
                          objective='binary:logistic',
                          use_label_encoder = False,
                          eval_set = eval_set
                          )

start = time.time()
modelfit = model.fit(X_train, y_train, eval_metric='logloss')
fittingTime = time.time() - start
start = time.time()
prediction = modelfit.predict(X_test)
InferenceTime = time.time() - start
F1score = f1_score(y_test, prediction)
probabilities = modelfit.predict_proba(X_test)
AUPRC = average_precision_score(y_test, probabilities[:, 1])
acc = accuracy_score(y_test, model.predict(X_test))

print('AUPRC = {}'.format(average_precision_score(y_test, probabilities[:, 1])))
print('F1 Score = {}'.format(F1score))
print('Fitting Time = {}'.format(fittingTime))
print('Inference Time = {}'.format(InferenceTime))
print('Accuracy = {}'.format(acc))

/Users/kartikullal/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[16:23:47] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/learner.cc:627: 
Parameters: { "eval_set" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


AUPRC = 0.813217980570374
F1 Score = 0.7115716753022453
Fitting Time = 0.9465470314025879
Inference Time = 0.008967876434326172
Accuracy = 0.984737707914458


In [191]:
import sigopt

# Create a connection to SigOpt using either your Development or API token
from sigopt import Connection

# put your token here
api_token = "SBGZRENFBLRQCATZZLETUQXQRQWMFBBHNAZTMKHMSBQEXIDK"
conn = Connection(client_token=api_token)

In [192]:
def create_model(assignments):
    model = xgb.XGBClassifier(
        n_estimators     = assignments['n_estimators'],
        min_child_weight = assignments['min_child_weight'],
        max_depth        = assignments['max_depth'],
        max_delta_step   = assignments['max_delta_step'],
        gamma            = assignments['gamma'],
        subsample        = assignments['subsample'],
        colsample_bytree = assignments['colsample_bytree'],
        colsample_bylevel = assignments['colsample_bylevel'],
        colsample_bynode = assignments['colsample_bynode'],
        reg_lambda       = assignments['lambda'],
        reg_alpha        = assignments['alpha'],
        learning_rate    = assignments['learning_rate'],
        

        # SigOpt-optimized parameters end here
        scale_pos_weight = 46,
        n_jobs = 4)
    return model

def evaluate_model(assignments):
    model = create_model(assignments)
    probabilities = model.fit(X_train, y_train).predict_proba(X_test)
    AUPRC = average_precision_score(y_test, probabilities[:, 1])
    return AUPRC

In [193]:
experiment = conn.experiments().create(
    name="Breast Cancer Prediction XGB - Vanilla SigOpt",

    parameters=[
        dict(name="n_estimators", bounds=dict(min=5,max=1000), type="int"),
        dict(name="min_child_weight", bounds=dict(min=1,max=15), type="int"),
        dict(name="max_depth", bounds=dict(min=2,max=20), type="int"),
        dict(name="max_delta_step", bounds=dict(min=0,max=10), type="int"),
        dict(name="gamma", bounds=dict(min=0,max=8), type="double"),
        dict(name="subsample", bounds=dict(min=0.5,max=1), type="double"),
        dict(name="colsample_bytree", bounds=dict(min=0.5,max=1), type="double"),
        dict(name="colsample_bylevel", bounds=dict(min=0.5,max=1), type="double"),
        dict(name="colsample_bynode", bounds=dict(min=0.5,max=1), type="double"),
        dict(name="lambda", bounds=dict(min=0.000001, max=1), type="double"),
        dict(name="alpha", bounds=dict(min=0.000001, max=1), type="double"),
        dict(name="learning_rate", bounds=dict(min=0.000001,max=1), type="double")
        ],

    metrics=[
        dict(name="AUPRC", objective="maximize", strategy="optimize")
        ],

    observation_budget = 250,
)

print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")

Explore your experiment: https://app.sigopt.com/experiment/576368/analysis


In [194]:
#Optimization Loop

for _ in range(experiment.observation_budget):

    suggestion = conn.experiments(experiment.id).suggestions().create()
    assignments = suggestion.assignments
    value = evaluate_model(assignments)

    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value
    )

    #update experiment object
    experiment = conn.experiments(experiment.id).fetch()

assignments = conn.experiments(experiment.id).best_assignments().fetch().data[0].assignments  

print("BEST ASSIGNMENTS \n", assignments)

BEST ASSIGNMENTS 
 Assignments({
  "alpha": 1e-06,
  "colsample_bylevel": 0.9311401178004796,
  "colsample_bynode": 1.0,
  "colsample_bytree": 0.530833373114606,
  "gamma": 3.185016597565008,
  "lambda": 0.026840256650179098,
  "learning_rate": 0.04742636535449742,
  "max_delta_step": 10,
  "max_depth": 20,
  "min_child_weight": 1,
  "n_estimators": 985,
  "subsample": 1.0
})


/Users/kartikullal/opt/anaconda3/lib/python3.8/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


In [197]:
model = XGBClassifier(reg_alpha = 0.000001,
                    learning_rate = 0.05,
                    max_depth = 20,
                    max_delta_step = 10,
                    n_estimators = 985,
                    scale_pos_weight=46,
                    colsample_bytree = 0.5,
                    colsample_bynode = 1,
                    colsample_bylevel = 0.9,
                    gamma = 3.2,
                    reg_lambda = 0.03,
                    min_child_weight = 1,
                    subsample = 1,
                    random_state=2)

In [198]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=0.9, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=3.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=10, max_depth=20, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=985,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=2,
              reg_alpha=1e-06, reg_lambda=0.03, ...)

In [199]:
y_pred = model.predict(X_test)


In [200]:
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
roc_auc = roc_auc_score(y_test,y_pred)
print('Precision Score', precision)
print('Recall Score', recall)
print('F1-Score', f1)
print('roc_auc_score', roc_auc)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Precision Score 0.7421602787456446
Recall Score 0.9181034482758621
F1-Score 0.8208092485549134
roc_auc_score 0.95559700891851
[[10636    74]
 [   19   213]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     10710
           1       0.74      0.92      0.82       232

    accuracy                           0.99     10942
   macro avg       0.87      0.96      0.91     10942
weighted avg       0.99      0.99      0.99     10942

